In [ ]:
# Please click on Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU
# Loading the libraries
import tensorflow as tf
import imutils
import threading
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications import vgg19
from tensorflow.keras.models import Model
from PIL import Image
import sys
from skimage.exposure import match_histograms, equalize_hist

In [ ]:
# Command for checking GPU in Colab
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-31dd7901-a7bb-a729-197f-3847bde1cfef)


In [ ]:
#Global Parameters
basewidth = baseheight = 512
IMAGE_DIRECTORY = "/content/"

In [ ]:
class LoadImages:
  def __init__(self, content_image_name, style_image_name):
    self.content_image = self.loading_img(content_image_name)
    self.style_image = self.loading_img(style_image_name)
  
  def load(self):
    return self.content_image, self.style_image

  def load_image_helper(self,im):
    im = cv2.resize(im,(basewidth,baseheight))
    im = im[...,::-1]
    return im

  def loading_img(self,image_name):
    if ('google.colab' in sys.modules):
      try:
        im = cv2.imread(IMAGE_DIRECTORY + image_name + ".jpeg",1)
        return self.load_image_helper(im)
      except:
        try:
          im = cv2.imread(IMAGE_DIRECTORY + image_name + ".jpg",1)
          return self.load_image_helper(im)
        except:
          try:
            im = cv2.imread(IMAGE_DIRECTORY + image_name + ".png",1)
            return self.load_image_helper(im)
          except:
            print("The image doesnt exist or the format isnt recognised.")
    else:
      try:
        im = cv2.imread(IMAGE_DIRECTORY + image_name + ".jpeg",1)
        return self.load_image_helper(im)
      except:
        try:
          im = cv2.imread(IMAGE_DIRECTORY + image_name + ".jpg",1)
          return self.load_image_helper(im)
        except:
          try:
            im = cv2.imread(IMAGE_DIRECTORY + image_name + ".png",1)
            return self.load_image_helper(im)
          except:
            print("The image doesnt exist or the format isnt recognised.")

In [ ]:
class InputImagePreprocessing:
  def __init__(self, enable_normal_method = False, enable_image_blending = True, enable_HSV = False, num_blocks = 4, laplacian_iterations = 5, CLAHE_gridsize = 8, CLAHE_clipLimit = 8.0):
    self.enable_normal_method = enable_normal_method
    self.enable_image_blending = enable_image_blending
    self.enable_HSV = enable_HSV
    self.num_blocks = num_blocks
    self.laplacian_iterations = laplacian_iterations
    self.CLAHE_gridsize = CLAHE_gridsize
    self.CLAHE_clipLimit = float(CLAHE_clipLimit)
  
  def preprocess(self, content_image, style_image):
    histogramMatchedImage = self.LocalizedHistogramMatching(content_image, style_image)
    clahe_adjusted_image = self.CLAHE_adjustment(histogramMatchedImage)
    return clahe_adjusted_image

  def CLAHE_adjustment(self, image):
    # To deal with the damaged contrast, perform CLAHE. Choose either HSV or LAB color space
    if self.enable_HSV == True:
      hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
      hsv_planes = cv2.split(hsv)
      gridsize = self.CLAHE_gridsize
      clahe = cv2.createCLAHE(clipLimit = self.CLAHE_clipLimit, tileGridSize = (gridsize,gridsize))
      hsv_planes[2] = clahe.apply(hsv_planes[2])
      hsv = cv2.merge(hsv_planes)
      clahe_adjusted_image = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    else:
      # The paper prefers CIELAB space
      lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
      lab_planes = cv2.split(lab)
      gridsize = self.CLAHE_gridsize
      clahe = cv2.createCLAHE(clipLimit = self.CLAHE_clipLimit, tileGridSize = (gridsize,gridsize))
      lab_planes[0] = clahe.apply(lab_planes[0])
      lab = cv2.merge(lab_planes)
      clahe_adjusted_image = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
    
    return clahe_adjusted_image

  def LocalizedHistogramMatching(self, content_image, style_image):
    if not os.path.exists('patches'):
      os.makedirs('patches')

    nRows = int(self.num_blocks/2)
    mCols = int(self.num_blocks/2)

    sizeX = content_image.shape[1]
    sizeY = content_image.shape[0]

    final_map = []
    for i in range(0,nRows):
      row_maps = []
      for j in range(0, mCols):
        # roi1 is the content_image being split and roi2 is style image
        roi1 = content_image[i*int(sizeY/nRows):i*int(sizeY/nRows) + int(sizeY/nRows) ,j*int(sizeX/mCols):j*int(sizeX/mCols) + int(sizeX/mCols)]
        roi2 = style_image[i*int(sizeY/nRows):i*int(sizeY/nRows) + int(sizeY/nRows) ,j*int(sizeX/mCols):j*int(sizeX/mCols) + int(sizeX/mCols)]
        cv2.imwrite('patches/content_patch_' + str(i) + str(j) + ".jpg", roi1)
        cv2.imwrite('patches/style_patch_' + str(i) + str(j) + ".jpg", roi2)
        # Perform histogram matching on the patch instead of global image
        roi = match_histograms(roi1, roi2, multichannel = True)
        row_maps.append(roi)
        
        cv2.imwrite('patches/matched_patch_'+str(i)+str(j)+".jpg", roi)
      if self.enable_image_blending == False:
        final_map.append(np.hstack(tuple(np.asarray(row_maps))))
      else:
        # join these two images using image blending
        final_map.append(self.LaplacianTransform(row_maps[0], row_maps[1]))

    if self.enable_image_blending == False:
      local_histogram_image = np.vstack(tuple(final_map))
    else:
      # join the two blocks vertically using image blending
      local_histogram_image = self.LaplacianTransform(final_map[0], final_map[1], horizontal=False)
    
    return local_histogram_image
  
  
  def LaplacianTransform(self, im1, im2, horizontal = True):
    iters = self.laplacian_iterations
    # generate Gaussian pyramid for im1
    im1_copy = im1.copy()
    gp_im1 = [im1_copy]
    for i in range(iters):
        im1_copy = cv2.pyrDown(im1_copy)
        gp_im1.append(im1_copy)

    # generate Gaussian pyramid for im2
    im2_copy = im2.copy()
    gp_im2 = [im2_copy]
    for i in range(iters):
        im2_copy = cv2.pyrDown(im2_copy)
        gp_im2.append(im2_copy)

    # generate Laplacian Pyramid for im1
    im1_copy = gp_im1[iters - 1]
    lp_im1 = [im1_copy]
    for i in range(iters - 1, 0, -1):
        gaussian_expanded = cv2.pyrUp(gp_im1[i])
        laplacian = cv2.subtract(gp_im1[i-1], gaussian_expanded)
        lp_im1.append(laplacian)

    # generate Laplacian Pyramid for im2
    im2_copy = gp_im2[iters - 1]
    lp_im2 = [im2_copy]
    for i in range(iters - 1, 0, -1):
        gaussian_expanded = cv2.pyrUp(gp_im2[i])
        laplacian = cv2.subtract(gp_im2[i-1], gaussian_expanded)
        lp_im2.append(laplacian)

    # Now add left and right halves of images in each level
    im1_im2_pyramid = []
    n = 0
    for im1_lap, im2_lap in zip(lp_im1, lp_im2):
        n += 1
        cols, rows, ch = im1_lap.shape
        if horizontal == True:
          laplacian = np.hstack((im1_lap, im2_lap))
        else:
          laplacian = np.vstack((im1_lap, im2_lap))
        im1_im2_pyramid.append(laplacian)
    # now reconstruct
    im1_im2_reconstruct = im1_im2_pyramid[0]
    for i in range(1, iters):
        im1_im2_reconstruct = cv2.pyrUp(im1_im2_reconstruct)
        im1_im2_reconstruct = cv2.add(im1_im2_pyramid[i], im1_im2_reconstruct)

    return im1_im2_reconstruct.astype('uint8')


In [ ]:
class NST:
  def __init__(self, input_image, content_image_name, content_image, style_image_name, style_image, STYLE_LAYERS, CONTENT_LAYERS, alpha = 1e-5, beta = 1e-2, optimizer_rate = 7, iterations = 200, enable_normalization = False, enable_normal_method = False):
    self.input_image = input_image
    self.content_image_name = content_image_name
    self.content_image = content_image
    self.style_image_name = style_image_name
    self.style_image = style_image
    self.alpha = alpha
    self.beta = beta
    self.optimizer_rate = optimizer_rate
    self.iterations = iterations
    self.STYLE_LAYERS = STYLE_LAYERS
    self.CONTENT_LAYERS = CONTENT_LAYERS
    self.enable_normalization = enable_normalization
    self.enable_normal_method = enable_normal_method
    
    model = vgg19.VGG19(weights='imagenet',input_shape=(basewidth,baseheight,3),include_top=False)
    model.trainable = False
    outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
    self.feature_extractor = Model(inputs=model.inputs, outputs=outputs_dict)

    self.content_and_style = []
    for content_layer,content_weight in self.CONTENT_LAYERS:
        self.content_and_style.append((self.feature_extractor(self.preprocess_img(content_image))[content_layer]))

    for style_layer,style_weight in self.STYLE_LAYERS:
        self.content_and_style.append((self.feature_extractor(self.preprocess_img(style_image))[style_layer]))
  
  def preprocess_img(self, img):
    if self.enable_normalization == False:
      img = img.astype('float32')
    else:
      img = img.astype('float32')/255.0

    if (tf.is_tensor(img) == False):
      if (len(img.shape) == 3):
        img = np.expand_dims(img,axis=0)
      img = tf.convert_to_tensor(img)
    return img
  
  def restore_img(self, img):
    if (tf.is_tensor(img) == True):
      img = img.numpy()
    img = np.squeeze(img,axis=0)
    if self.enable_normalization == False:
      return img
    else:
      return img * 255.0

  def content_cost(self, a_C, a_G):
    return tf.reduce_sum(tf.square(tf.subtract(a_C, a_G)))/2

  def gram_matrix(self, A):
    A = tf.reshape(A, (-1, A.shape[-1])) 
    gram_matrix = tf.matmul(A, A, transpose_a=True)
    return gram_matrix

  def style_cost(self, a_S, a_G):
    return tf.reduce_sum(tf.square(tf.subtract(self.gram_matrix(a_S), self.gram_matrix(a_G))))

  def iteration_loss(self, new_img):
    generated_image_model = self.feature_extractor(new_img)  
    content_loss = 0
    style_loss = 0
    track = 0
 
    for content_layer, content_weight in self.CONTENT_LAYERS:
      content_loss += content_weight * self.content_cost((generated_image_model[content_layer]), self.content_and_style[track])
      track = track + 1
    
    for style_layer, style_weight in self.STYLE_LAYERS:
      style_loss += style_weight * self.style_cost((generated_image_model[style_layer]), self.content_and_style[track])
      track = track + 1
    
    tv_loss = tf.reduce_sum(tf.image.total_variation(new_img))
    if self.enable_normal_method == True:
      self.normal_content_losses.append(self.alpha * content_loss)
      self.normal_style_losses.append(self.beta * style_loss)
    else:
      self.local_content_losses.append(self.alpha * content_loss)
      self.local_style_losses.append(self.beta * style_loss)
    
    total_loss = self.beta * style_loss + self.alpha * content_loss + tv_loss
    return total_loss

  def plotTotalLoss(self):
    plt.plot((self.normal_losses),label='Normal method')
    plt.plot((self.local_losses),label='Proposed method')
    plt.xlabel('Iterations')
    plt.ylabel('Log (Total Loss)')
    plt.legend()
    plt.savefig('Total_loss.jpeg')
    plt.close()
  
  def plotContentLoss(self):
    plt.plot(np.log10(self.normal_content_losses),label='Normal method')
    plt.plot(np.log10(self.local_content_losses),label='Proposed method')
    plt.xlabel('Iterations')
    plt.ylabel('Log (Content Loss)')
    plt.legend()
    plt.savefig('Content_loss.jpeg')
    plt.close()
  
  def plotStyleLoss(self):
    plt.plot(np.log10(self.normal_style_losses),label='Normal method')
    plt.plot(np.log10(self.local_style_losses),label='Proposed method')
    plt.xlabel('Iterations')
    plt.ylabel('Log (Style Loss)')
    plt.legend()
    plt.savefig('Style_loss.jpeg')
    plt.close()

  def double_mode(self):
    self.single_mode(normal_method = True)
    self.single_mode(normal_method = False)

  def single_mode(self, normal_method = False, print_loss = False):
    
    if normal_method == False:
      self.enable_normal_method = False
      self.local_losses = []
      self.local_content_losses = []
      self.local_style_losses = []
    else:
      self.enable_normal_method = True
      self.normal_losses = []
      self.normal_content_losses = []
      self.normal_style_losses = []
      
    if self.enable_normal_method == True:
      new_img = self.content_image
    else:
      new_img = self.input_image 

    if self.enable_normalization == False:
      new_img = tf.Variable(self.preprocess_img(new_img))
    else:
      new_img = tf.Variable(self.preprocess_img(new_img))
    
    opt = tf.optimizers.Adam(self.optimizer_rate)
    for i in range(1,self.iterations + 1):
      with tf.GradientTape(persistent = False) as g:
        g.watch(new_img)
        total_loss = self.iteration_loss(new_img)
        if print_loss == True:
          print("Iteration: " + str(i) + " Loss: " + str(total_loss.numpy()))
        grad = g.gradient(total_loss, new_img)
        opt.apply_gradients(zip([grad],[new_img]))

        if self.enable_normal_method == True:
          self.normal_losses.append(np.log10(total_loss.numpy()))
        else:
          self.local_losses.append(np.log10(total_loss.numpy()))

        if self.enable_normalization == False:
          new_img.assign(tf.clip_by_value(new_img, 0.0, 255.0))
        else:
          new_img.assign(tf.clip_by_value(new_img, 0.0, 1.0))

        if (i % 20 == 0):
          generated_image = self.restore_img(new_img)
          im = Image.fromarray(generated_image.astype('uint8'),'RGB')
          if normal_method == False:           
            im.save("op_" + str(i) + "_local.png")
          else:
            im.save("op_" + str(i) + "_normal.png")
      
      generated_image = self.restore_img(new_img)
      # Convert the image to Image object so that save method can be used on it
      output_image = Image.fromarray(generated_image.astype('uint8'),'RGB')
      if normal_method == False:
        output_image.save(self.content_image_name + "+" + self.style_image_name + "+histogram+local.png")
      else:
        output_image.save(self.content_image_name + "+" + self.style_image_name + "+histogram+normal.png")

In [ ]:
def main():
  # Images are loaded in RGB format
  content_image_name="dielit"
  style_image_name="johnson_style"
  content_image, style_image = LoadImages(content_image_name = content_image_name,
                                          style_image_name = style_image_name).load()
  input_image_preprocessor = InputImagePreprocessing()
  input_image = input_image_preprocessor.preprocess(content_image, style_image)
  STYLE_LAYERS = [
    ('block1_conv1', 0.2),
    ('block2_conv1', 0.2),
    ('block3_conv1', 0.2),
    ('block4_conv1', 0.2),
    ("block5_conv1",0.2)]
  CONTENT_LAYERS = [
    ("block4_conv1",1)]
  nst = NST(input_image, content_image_name, content_image,
            style_image_name, style_image, STYLE_LAYERS, CONTENT_LAYERS)
  # nst.double_mode()
  # nst.plotTotalLoss()
  # nst.plotContentLoss()
  # nst.plotStyleLoss()

  nst.single_mode()
  
if __name__=="__main__":
  main()